In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install evaluate
!pip install rouge


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [3]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
data=pd.read_csv('/content/QA_Data.csv')

In [5]:
data.head()

,Unnamed: 0,context,question,answers
0,0,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenirdis I?,Amenirdis I was a God's Wife of Amun during th...
1,1,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenhotep III?,"Amenhotep III, also known as Amenhotep the Mag..."
2,2,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Khafre?,"Khafre, also known as Khephren or Chephren, wa..."
3,3,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenemhat III?,"Amenemhat III, also known as Amenemhet III, wa..."
4,4,\nAmenirdis I (throne name: Hatneferumut) was ...,What was discovered by Professor Maspero in Th...,A coffin and mummy of a gazelle from the 21st ...


In [6]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answers']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [7]:

# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [8]:
import torch
from tqdm.auto import tqdm

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MODEL = MODEL.to(DEVICE)

train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(50):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch.get("decoder_attention_mask", None)
        if decoder_attention_mask is not None:
            decoder_attention_mask = decoder_attention_mask.to(DEVICE)

        outputs = MODEL(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()

        train_loss += outputs.loss.item()
        train_batch_count += 1

    MODEL.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation batches"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            decoder_attention_mask = batch.get("decoder_attention_mask", None)
            if decoder_attention_mask is not None:
                decoder_attention_mask = decoder_attention_mask.to(DEVICE)

            outputs = MODEL(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                decoder_attention_mask=decoder_attention_mask
            )

            val_loss += outputs.loss.item()
            val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")



Training batches:   0%|          | 0/77 [00:00<?, ?it/s]

Validation batches:   0%|          | 0/20 [00:00<?, ?it/s]

1/2 -> Train loss: 3.2154287146283433	Validation loss: 2.3981557548046113


Training batches:   0%|          | 0/77 [00:00<?, ?it/s]

Validation batches:   0%|          | 0/20 [00:00<?, ?it/s]

...

Validation batches:   0%|          | 0/20 [00:00<?, ?it/s]

49/2 -> Train loss: 1.6713324437861319	Validation loss: 1.07977083700576


Training batches:   0%|          | 0/77 [00:00<?, ?it/s]

Validation batches:   0%|          | 0/20 [00:00<?, ?it/s]

50/2 -> Train loss: 1.6563500870067578	Validation loss: 1.0658987679295242


In [9]:
MODEL.save_pretrained("/content/drive/MyDrive/T5/qa_model")
TOKENIZER.save_pretrained("/content/drive/MyDrive/T5/qa_tokenizer")

('/content/drive/MyDrive/T5/qa_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/T5/qa_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/T5/qa_tokenizer/spiece.model',
 '/content/drive/MyDrive/T5/qa_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/T5/qa_tokenizer/tokenizer.json')

# inference

In [10]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0).to(DEVICE)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0).to(DEVICE)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer], references=[[ref_answer]])

        print("Context:\n", context)
        print("\nQuestion:\n", question)
        print("\nReference Answer:\n", ref_answer)
        print("\nPredicted Answer:\n", predicted_answer)
        print("\nBLEU Score:\n", score['google_bleu'])

    else:
        print("\nPredicted Answer:\n", predicted_answer)



In [11]:
context = """

Amenirdis I (throne name: Hatneferumut) was a God's Wife of Amun during the 25th Dynasty of ancient Egypt. Originating from the Kingdom of Kush, she was the daughter of Pharaoh Kashta and Queen Pebatjma, and was later adopted by Shepenupet I. She went on to rule as high priestess, and has been shown in several artifacts from the period.

== Biography ==
She was a Kushite princess, the daughter of Pharaoh Kashta and Queen Pebatjma. She is likely to have been the sister of pharaohs Shabaka and Piye. Kashta arranged to have Amenirdis I adopted by the Divine Adoratrice of Amun, Shepenupet I, at Thebes as her successor. This shows that Kashta already controlled Upper Egypt prior to the reign of Piye, his successor.She ruled as high priestess approximately between 714 and 700 BCE, under the reigns of Shabaka and Shabataka, and she adopted Piye's daughter Shepenupet II as her successor. She also held the priestly titles of Divine Adoratrice of Amun and God's Hand. Upon her death, she was buried in a tomb in the grounds of Medinet Habu.She is depicted in the Osiris-Hekadjet ("Osiris, Ruler of Eternity") temple in the Karnak temple complex, and in Wadi Gasus, along with Shepenupet I. She is mentioned on two offering tables, five statues, a stela and several small objects including scarabs. A statue of Amenirdis I carved from granitoid and decorated in gold leaf is held by the Nubian Museum in Aswan, Upper Egypt. The statue itself shows her decorated in the Egyptian style, with similarities to depictions of Isis and Hathor.

"""
question = "Who was Amenirdis I?"
ref_answer = "Amenirdis I was a God's Wife of Amun during the 25th Dynasty of ancient Egypt."

predict_answer(context, question, ref_answer)


Context:
 

Amenirdis I (throne name: Hatneferumut) was a God's Wife of Amun during the 25th Dynasty of ancient Egypt. Originating from the Kingdom of Kush, she was the daughter of Pharaoh Kashta and Queen Pebatjma, and was later adopted by Shepenupet I. She went on to rule as high priestess, and has been shown in several artifacts from the period.

== Biography ==
She was a Kushite princess, the daughter of Pharaoh Kashta and Queen Pebatjma. She is likely to have been the sister of pharaohs Shabaka and Piye. Kashta arranged to have Amenirdis I adopted by the Divine Adoratrice of Amun, Shepenupet I, at Thebes as her successor. This shows that Kashta already controlled Upper Egypt prior to the reign of Piye, his successor.She ruled as high priestess approximately between 714 and 700 BCE, under the reigns of Shabaka and Shabataka, and she adopted Piye's daughter Shepenupet II as her successor. She also held the priestly titles of Divine Adoratrice of Amun and God's Hand. Upon her death, 